In [1]:
from __future__ import print_function, division
from builtins import range, input

In [2]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from datetime import datetime

In [11]:
from keras.models import Model
from keras.layers import Input, Embedding, Flatten, Dense, Concatenate
from keras.layers import Dropout, BatchNormalization, Activation
from keras.regularizers import l2
from keras.optimizers import SGD, Adam

In [5]:
df = pd.read_csv('../Data/edited_rating.csv')

In [6]:
N = df.userId.max() + 1 # number of users
M = df.movie_idx.max() + 1 # number of movies

In [7]:
df = shuffle(df)
cutoff = int(0.8 * len(df))
df_train = df.iloc[:cutoff]
df_test = df.iloc[cutoff:]

In [8]:
K=10
mu = df_train.rating.mean()
epochs = 25
reg = 0. # regularization penalty

In [12]:
# keras model
# note that batch size is implicit, so this means our input is batch sized by 1
# shape parameter specifies the input shape of the layer
u = Input(shape=(1,))
m = Input(shape=(1,)) 
u_embedding = Embedding(N, K,embeddings_regularizer=l2(reg))(u) # (N, 1, K)
m_embedding = Embedding(M, K,embeddings_regularizer=l2(reg))(m) # (M, 1, K)
# In particular, the output of this is not batch sized by k, instead it is batch sized by 1 x k
u_embedding = Flatten()(u_embedding) # (N, K)
m_embedding = Flatten()(m_embedding) # (M, K)
x=Concatenate()([u_embedding,m_embedding])

user_ids.shape: (5,)
1/1 [==============================] - 0s 300ms/step
p[0].shape: (5, 1, 10)
p[1].shape: (5, 1, 10)


In [13]:
# the neural network
x = Dense(400)(x)
# x = BatchNormalization()(x)
x = Activation('relu')(x)
# x = Dropout(0.5)(x)
# x = Dense(100)(x)
# x = BatchNormalization()(x)
# x = Activation('relu')(x)
x = Dense(1)(x)


In [14]:
model = Model(inputs=[u, m], outputs=x)
model.compile(
    loss='mse',
    optimizer=SGD(lr=0.01,momentum=0.9),
    metrics=['mse'],
)

c:\Users\mrizv\anaconda3\lib\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [15]:
r=model.fit(
    x=[df_train.userId.values, df_train.movie_idx.values],
    y=df_train.rating.values - mu,
    epochs=epochs, 
    batch_size=128,
    validation_data=(
        [df_test.userId.values, df_test.movie_idx.values],
        df_test.rating.values - mu
    )
)

Epoch 1/25
  1543/125002 [..............................] - ETA: 28:21 - loss: 1.0450 - mse: 1.0450

KeyboardInterrupt: 

In [ ]:
# plot losses
plt.plot(r.history['loss'], label="train loss")
plt.plot(r.history['val_loss'], label="test loss")
plt.legend()
plt.show()

#plot mse
plt.plot(r.history['mean_squared_error'], label="train mse")
plt.plot(r.history['val_mean_squared_error'], label="test mse")
plt.legend()
plt.show()